In [1]:
import pymongo, os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from collections import OrderedDict
from com.ibm.watson.umls.umls_db import Mrconso

/Users/singhv/Library/Python/2.7/lib/python/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
client = pymongo.MongoClient('bioresearch-d1', 23001)
db = client["WATSON"]

In [3]:
DESKTOP = os.path.join(os.environ["HOME"], "Desktop")
base_dir = os.path.join(DESKTOP, "trial_stats")
#targeted_therapy_trials = [ line.strip() for line in open(os.path.join(base_dir, "targeted_therapy_ncts.txt")) if line != "" ]
#existing_trials = [ line.strip() for line in open(os.path.join(base_dir, "existing_trials_wfg.txt")) if line != "" ]
#cancer_genes = pd.read_csv(os.path.join(base_dir, "cancer_genes50.txt"),header=None)[0]
#cancer_genes = list(cancer_genes)
#new_targeted_therapy_trials = set(targeted_therapy_trials).difference(set(existing_trials))

In [4]:
# Read Jia's table containing relevant drugs DRUG ID DRUG_NAME
therapy_table = pd.read_excel(os.path.join(base_dir, "Drugs_to_query_for_trials.xlsx"), index_col=0)
print therapy_table.head(10)

                                                  Therapy in Quest priory list
Therapy_ID                                                                    
3058                                           Crenolanib                    Y
882                                 CDK_Inhibitor_P276-00                  NaN
5127        Anti-TGF-beta_RII_Monoclonal_Antibody_IMC-TR1                  NaN
5075                              MDM2_Inhibitor_DS-3032b                  NaN
5176                                PI3K_Inhibitor_PX-866                  NaN
1070                                          Saracatinib                  NaN
4098                              c-Kit_Inhibitor_PLX9486                  NaN
4074                      mTOR1/2_Kinase_Inhibitor_ME-344                  NaN
700                                           Lysofylline                  NaN
156                                            Nintedanib                    Y


In [5]:
# Querying - if querying based on index don't use the column name instead use DataFrame.index
#print therapy_table[ therapy_table['Therapy'] == 'Vorinostat' ]
print (therapy_table.index == "DB02546").any()

True


In [ ]:
# Reading WGA_ALL_DRUG_GENERAL table from Dev environment dump
wga_drugdb_dev_table = pd.read_csv(os.path.join(base_dir, "WGA_ALL_DRUG_GENERAL3.tsv"), 
                                   delimiter="\t", header=None,
                                   names=['THERAPY_ID', 'THERAPY', 'NCI_CODE', 'NCI_MOA', 'NCI_CUI', 'GENERIC_NAME', 
                                          'FDA_STATUS', 'ALIAS_NAME', 'CODE_NAME', 'ACTIVE', 'RETIRE_DATE', 
                                          'MANUALLY_REVIEWED', 'FDA_LABEL_INDICATIONS_AND_USAGE', 'CREATED', 
                                          'THERAPY_SHORT_NAME', 'UMLS_CUI'])
#print wga_drugdb_dev_table.head(10)
wga_drugdb_dev_table['NCI_CUI'].fillna("missing")

In [ ]:
# Adding NCI_CODE, NCIM_CODE and UMLS_CUI columns to Jia's spreadsheet of relevant drugs
therapy_to_nci_dict = dict()
therapy_to_ncim_dict = dict()
therapy_to_umls_dict = dict()
for index in therapy_table.index.values:
    match = (wga_drugdb_dev_table['THERAPY_ID'] == str(index))
    if match.any():
        ncim_cui = wga_drugdb_dev_table[match]["NCI_CUI"].values[0]
        nci_code = wga_drugdb_dev_table[match]["NCI_CODE"].values[0]
        print "%s: %s" % (index, ncim_cui)
        if (ncim_cui is not None) and isinstance(ncim_cui, str) and ("CL" in ncim_cui):
            result = Mrconso.select().where( (Mrconso.code == nci_code) & (Mrconso.sab.contains("NCI")) )   
            therapy_to_umls_dict[index] = result.get().cui
        else:
            therapy_to_umls_dict[index] = ncim_cui
        therapy_to_nci_dict[index] = nci_code
        therapy_to_ncim_dict[index] = ncim_cui
    else:
        therapy_to_nci_dict[index] = ""
        therapy_to_ncim_dict[index] = ""
        
therapy_table["NCI_CODE"] = pd.Series(therapy_to_nci_dict)
therapy_table["NCIM_CUI"] = pd.Series(therapy_to_ncim_dict)
therapy_table["UMLS_CUI"] = pd.Series(therapy_to_umls_dict)
print therapy_table.head(2)

In [ ]:
#print wga_drugdb_dev_table[wga_drugdb_dev_table['THERAPY_ID'] == '3203']["NCI_CODE"]
#print wga_drugdb_dev_table[wga_drugdb_dev_table['NCI_CUI'].str.contains("nib", case=False)]
print therapy_table[ therapy_table['Therapy'] == 'Danusertib' ]
therapy_table.to_excel(os.path.join(base_dir, "Drugs_to_query_for_trials_MODIFIED.xlsx"))

In [2]:
result = Mrconso.select().where( (Mrconso.cui == "C0732355") & (Mrconso.sab.contains("NCI")) &
                               (Mrconso.tty == "PT") & (Mrconso.isPref == "Y"))
if result:
    print result.get().str

In [ ]:
#quest_drugs = pd.read_excel(os.path.join(base_dir, "Drugs_for_Quest_Panel.tsv"))
#new_studies = db["CTGOV"].find({"NCT_ID" : {"$in" : list(new_targeted_therapy_trials)}, "COUNTRIES" : "United States"})
new_studies = db["CTGOV"].find({"metadata.STATUS" : "NEW", "STATUS" : {"$ne" : "Terminated"}, "COUNTRIES" : "United States"})
all_drug_to_trial_dicts = []
studies_count_based_on_drug = dict()
for study in new_studies:
    temp = []
    for intervention in study["interventions"]:
        nci_code = None
        wga_drug_id = intervention.get("WGADB_ID")
        if "ID" in intervention:
            result = Mrconso.select().where( (Mrconso.cui == intervention["ID"]) & (Mrconso.sab.contains("NCI")) )
            if result:
                nci_code = result.get().code
        if (wga_drug_id is not None) or (nci_code is not None):
            temp.append( (wga_drug_id, nci_code, study["NCT_ID"]) )              
    all_drug_to_trial_dicts.extend(temp)
    
print all_drug_to_trial_dicts[:10]

In [ ]:
for (wga_drug_id, nci_code, nct_id) in all_drug_to_trial_dicts:
    if ( (therapy_table["NCI_CODE"] == nci_code).any() ) or ( (therapy_table.index == wga_drug_id).any() ):
        if wga_drug_id in studies_count_based_on_drug:
            studies_count_based_on_drug[wga_drug_id].append(nct_id)
        else:
            studies_count_based_on_drug[wga_drug_id] = [nct_id]

In [ ]:
country_counts = dict()
for study in db["CTGOV"].find({"NCT_ID" : {"$in" : targeted_therapy_trials}}):
    for country in set(study["COUNTRIES"]):
        if country in country_counts: 
            country_counts[country].append(study["NCT_ID"])
        else:
            country_counts[country] = [study["NCT_ID"]]
            
country_counts = OrderedDict(sorted(country_counts.items(), key=lambda x: len(x[1]), reverse=True))

In [ ]:
top10_countries = [ x[0] for x in country_counts[:10] ]
top10_count = [ len(x[1]) for x in country_counts[:10] ]
print top10_count

In [ ]:
ind = np.arange(len(country_counts))
#print ind
width = 0.35
plt.figure(figsize=(20,10))
rects = plt.bar(ind, [len(x) for x in country_counts.values()])
plt.xticks(ind + width, country_counts.keys(), rotation='vertical')
plt.show()

In [ ]:
temp = []
[ temp.extend(value) for key, value in studies_count_based_on_drug.iteritems() if key is not None  ]
print len(set(temp))
#db["CTGOV"].update_many({"NCT_ID" : {"$in" : temp}}, {"$set" : {"metadata.REVIEWER" : "pmcneil"}})
#with open(os.path.join(base_dir, "all_quest_trial_Nov17.txt"), "wb") as f:
#    [f.write(nct_id + "\n") for nct_id in set(temp)]
#print studies_count_based_on_drug["DB08916"]
with open(os.path.join(base_dir, "all_inscope_drug_trials.tsv"), "wb") as f:
    for drug_id in studies_count_based_on_drug.keys():
        #print drug_id
        if drug_id is not None:
            f.write("\t".join([drug_id] + list(set(studies_count_based_on_drug.get(drug_id, []))) + ["\n"]))

In [ ]:
studies_count_based_on_drug = OrderedDict(sorted(studies_count_based_on_drug.items(), key=lambda t: len(t[1])))
ind = np.arange(len(studies_count_based_on_drug))
width = 0.35
plt.figure(figsize=(20,10))
rects = plt.bar(ind, [ len(x) for x in studies_count_based_on_drug.values() ])
plt.xticks(ind + width, studies_count_based_on_drug.keys(), rotation='vertical')
plt.xlabel('WGA Drug ID', fontsize=18)
plt.ylabel('Number of Trials', fontsize=16)
plt.show()

In [ ]:
df = pd.DataFrame({"NCT_IDS": [", ".join(nct_ids) for nct_ids in studies_count_based_on_drug.values()]}, index=studies_count_based_on_drug.keys())
print studies_count_based_on_drug.keys()

In [ ]:
df.to_csv(os.path.join(base_dir,"quest_drug_trials_v2.tsv"), sep="\t")

In [ ]:
lead_sponsors_dict = dict()
collaborators_dict = dict()
for study in db["CTGOV"].find({"NCT_ID" : {"$in" : list(new_targeted_therapy_trials)}, "COUNTRIES" : "United States"}):
    lead_sponsor = study["leadSponsor"]["AGENCY"]
    lead_sponsors_dict[lead_sponsor] = lead_sponsors_dict.get(lead_sponsor, 0) + 1
    for collaborator in study["collaborators"]:
        collaborators_dict[collaborator["AGENCY"]] = collaborators_dict.get(collaborator["AGENCY"], 0) + 1
        
lead_sponsors_dict = OrderedDict(sorted(lead_sponsors_dict.items(), key=lambda t: t[1], reverse=True))
ind = np.arange(50)
width = 0.35
plt.figure(figsize=(20,10))
rects = plt.bar(ind, lead_sponsors_dict.values()[:50])
plt.xticks(ind + width, lead_sponsors_dict.keys()[:50], rotation='vertical')
plt.xlabel('Lead Sponsor', fontsize=18)
plt.ylabel('Number of Trials', fontsize=16)
plt.show()

In [ ]:
collaborators_dict = OrderedDict(sorted(collaborators_dict.items(), key=lambda t: t[1], reverse=True))
ind = np.arange(50)
width = 0.35
plt.figure(figsize=(20,10))
rects = plt.bar(ind, collaborators_dict.values()[:50])
plt.xticks(ind + width, collaborators_dict.keys()[:50], rotation='vertical')
plt.xlabel('Collaborator', fontsize=18)
plt.ylabel('Number of Trials', fontsize=16)
plt.show()